In [21]:
### import packages for authentication

from google.cloud import bigquery
from google.oauth2 import service_account

### import packages for converting query results into dataframe

import pandas as pd 

In [22]:
### authentication 

credentials = service_account.Credentials.from_service_account_file("/Users/HenriekeMax/Documents/Career_Development/GitHub/FrauenLoop_NLP_Project_2020/data/raw/GoogleBigQuery_key.json")


In [23]:
### construct a BigQuery client object.

client = bigquery.Client(credentials=credentials, 
project = credentials.project_id)

In [24]:
### getting overview of Stack Overflow tables

stackoverflow = client.dataset('stackoverflow', project= 'bigquery-public-data')
print([x.table_id for x in client.list_tables(stackoverflow)])

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


In [25]:
### make an API request

query = """
SELECT
      pq.id as question_id, pa.parent_id as question_id_check, pq.title as question_title, pq.body as question_text, pq.accepted_answer_id, pq.answer_count, pq.comment_count, pq.community_owned_date,
      pq.creation_date, pq.favorite_count, pq.last_activity_date, pq.last_edit_date, pq.last_editor_display_name, 
      pq.last_editor_user_id, pq.owner_display_name, pq.owner_user_id, pq.post_type_id, pq.score,
      pq.tags, pq.view_count,
      pa.id as answer_id, pa.body as answer_text
FROM `bigquery-public-data.stackoverflow.posts_questions` pq
INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` pa ON pq.id = pa.parent_id
WHERE pa.creation_date > "2019-05-30 00:00:00.000 UTC"
"""

### query_job = client.query(query) 


### df = client.query(query).to_dataframe()

dataframe = (
    client.query(query)
    .result()
    .to_dataframe()
)

In [26]:
### display query results

display(dataframe)

,question_id,question_id_check,question_title,question_text,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,...,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,post_type_id,score,tags,view_count,answer_id,answer_text
0,58830869,58830869,django checkbox yes or no,"<pre><code> &lt;form id=""form"" name=""form""&gt...",59168090.0,2,4,NaT,2019-11-13 05:33:13.727000+00:00,NaN,...,user12324446,NaN,user12324446,NaN,1,0,javascript|django,82,59168090,"<p>you dont need javascript for that,</p>\n\n<..."
1,46307932,46307932,Network Error in axios post request react-native,<p>This is my method to checking Auth by axios...,NaN,6,6,NaT,2017-09-19 18:57:42.003000+00:00,1.0,...,None,6411354.0,None,6411354.0,1,0,react-native|axios,7779,60026866,<p>Add <code>http://</code> to your url. If it...
2,60400749,60400749,"Using correct terminology: What is a ""class re...",<p>In an effort to ensure the use of proper te...,60400809.0,3,14,NaT,2020-02-25 18:06:12.303000+00:00,NaN,...,None,1345535.0,None,12095573.0,1,5,c#|oop,105,60400871,"<p>As stated, no - <code>myObjectTest</code> r..."
3,60400749,60400749,"Using correct terminology: What is a ""class re...",<p>In an effort to ensure the use of proper te...,60400809.0,3,14,NaT,2020-02-25 18:06:12.303000+00:00,NaN,...,None,1345535.0,None,12095573.0,1,5,c#|oop,105,60400809,"<p>I usually say ""myObjectTest is a variable o..."
4,35327679,35327679,Validate or remove for extra fields in laravel,<p>It's posible to validate request with rules...,NaN,11,2,NaT,2016-02-10 22:59:18.777000+00:00,4.0,...,None,NaN,None,1958210.0,1,21,php|forms|laravel,29347,57009561,<pre><code>$request-&gt;validated(); \n</code>...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1789025,58132624,58132624,How can I order formatted (MMM yyyy) date?,<p>I tried to change my date format from <code...,58134485.0,1,1,NaT,2019-09-27 10:22:05.733000+00:00,NaN,...,None,NaN,None,11495854.0,1,1,google-bigquery,37,58134485,<p>Below is for BigQuery Standard SQL </p>\...
1789026,56815815,56815815,Winston 3.x support for logstash missing?,<p>I'm trying to integrate a dummy nodejs app ...,56919159.0,1,0,NaT,2019-06-29 07:23:05.597000+00:00,NaN,...,None,NaN,None,1494134.0,1,0,node.js|logging|logstash|winston|elk,1096,56919159,"<p>There are two concepts at play here, in Win..."
1789027,56498312,56498312,MongoDB _id returns undefined on frontend (VueJS),<p>I created a simple fullstack webapp using N...,NaN,2,0,NaT,2019-06-07 16:44:24.477000+00:00,NaN,...,None,NaN,None,9786502.0,1,0,javascript|node.js|mongodb|vue.js,74,56499650,<p>You need to add the property reference <cod...
1789028,1040402,1040402,removing html element styles via javascript,<p>I'm trying to replace an element's inline s...,1040439.0,6,3,NaT,2009-06-24 19:22:47.503000+00:00,12.0,...,user1228,NaN,None,91748.0,1,86,javascript|html|dom|styles,187090,56648232,"<p>In jQuery, you can use</p>\n\n<pre><code>$(..."


In [27]:
### save dataframe to a csv file

dataframe.to_csv('/Users/HenriekeMax/Documents/Career_Development/GitHub/FrauenLoop_NLP_Project_2020/data/raw/stackoverflow_raw.csv')

In [ ]:
### some code here about experimenting with the retry-parameter in case rate limit is exceeded
### did not use this, just found and wanted to keep for now in case it was needed

from google.cloud.biquery import DEFAULT_RETRY

query_deadline = 5 * 60  # seconds
query_retry = DEFAULT_RETRY.with_deadline(query_deadline)
job = client.query(query, retry=query_retry)